<a href="https://colab.research.google.com/github/venkeeg/Python-codes/blob/master/73strings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab  import files
# Upload following files: Occupation Data.xlsx,Skills.xlsx,Technology Skills.xlsx, Tools Used.xlsx
uploaded = files.upload()

In [ ]:
import pandas as pd
#Read Files Occupation Data,Skills. In Skills file retain only those skills whose importance is 
#greater than a threshold value. 
df_occ = pd.read_excel(r'Occupation Data.xlsx')
df_skls = pd.read_excel(r'Skills.xlsx',usecols = ['Title','Element Name','Scale ID','Data Value'])
df_skls = df_skls.loc[((df_skls['Data Value']>4.0) & (df_skls['Scale ID']=='IM'))]
df_skls.drop(['Scale ID','Data Value'],axis=1,inplace=True)
 
#Grouping the dataframe by Title column so that all skills for a particular Title are
#arranged as columns.
s = df_skls.groupby('Title').cumcount().add(1)
df_skls = (df_skls.set_index(['Title',s]).unstack().sort_index(axis=1,level=1))
df_skls.columns = ['{}_{}'.format(a, b) for a,b in df_skls.columns]
df_skls = df_skls.reset_index()
df_skls.dropna(axis=1,how='all',inplace=True)

 
#Read Technology skills file. Retain only those entries which are hot skills
df_tecskls = pd.read_excel(r'Technology Skills.xlsx',usecols=['Title','Example','Hot Technology'])
#df_tecskls = df_tecskls.loc[(df_tecskls['Hot Technology']=='Y')]
df_tecskls.drop(['Hot Technology'],axis=1,inplace=True)
 
#Grouping by Title and arranging tech skills as columns
t = df_tecskls.groupby('Title').cumcount().add(1)
df_tecskls = (df_tecskls.set_index(['Title',t]).unstack().sort_index(axis=1,level=1))
df_tecskls.columns = ['{}_{}'.format(a, b) for a,b in df_tecskls.columns]
df_tecskls = df_tecskls.reset_index()
df_tecskls.dropna(axis=1,how='all',inplace=True)

 
#Read 'Tools Used' file. 
df_tools = pd.read_excel(r'Tools Used.xlsx',usecols=['Title','Example'])

#rename column name 'Example' to 'Tools'
df_tools.rename(columns={'Example':'Tools'},inplace=True) 
#grouping by Title and arranging Tools uses as columns
u = df_tools.groupby('Title').cumcount().add(1)
df_tools = (df_tools.set_index(['Title',u]).unstack().sort_index(axis=1,level=1))
df_tools.columns = ['{}_{}'.format(a, b) for a,b in df_tools.columns]
df_tools = df_tools.reset_index()
df_tools.dropna(axis=1,how='all',inplace=True)
 
#Merging 3 data frames by performing an outer join on 'Title' column
df_skills = pd.merge(df_tools,df_tecskls,on='Title',how='outer')
df_skills = pd.merge(df_skills,df_skls,on='Title',how='outer')
df_skills.dropna(axis=1,how='all',inplace=True)

#Remove columns which have very few rows
cols =[]
for col,count in dict(df_skills.count()).items():
  if count<10:
    cols.append(col)

df_skills.drop(columns=cols,inplace=True)

#Replacing NaNs with blank spaces
df_skills.fillna("",inplace=True)

#Store Job title and skills in different dataframes
X=df_skills['Title'].copy()
y = df_skills.copy()
y.drop(columns=['Title'],inplace=True)

#Import necessary modules
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import KFold
import re
import openpyxl

#Pipeline to vectorize and build Job Title-> Skills prediction model
pipeline1 = Pipeline([('cvr', CountVectorizer(stop_words='english',ngram_range=(1,2))),
                     ('clf', MultiOutputClassifier(MultinomialNB()))])

#Convert y to numpy array for compatibility with binarizer
y=y.to_numpy()

#Binarize the labels using Multilabel binarizer
mulbi = MultiLabelBinarizer()
y= mulbi.fit_transform(y)

#Convert y(labels) back to dataframe
y = pd.DataFrame(y)

#Performing KFold cross validation
kf = KFold(n_splits=10,shuffle=True)

for train_idx,test_idx in kf.split(X,y):
  X_train,X_test = X.iloc[train_idx],X.iloc[test_idx]
  y_train,y_test = y.iloc[train_idx],y.iloc[test_idx]
  #Train the model
  print("Training model 1...")
  pipeline1.fit(X_train, y_train)
  #Predict
  pred = pipeline1.predict(X_test)


################## Processing the file containing job description and title ########################

#Store description and Job title if different Dataframes
X_jd = df_occ['Description'].copy()
y_jd = df_occ['Title'].copy()

#Pipeline to vectorize and build Description->Job Title prediction model
pipeline2 = Pipeline([('cvr',TfidfVectorizer(stop_words='english',ngram_range=(1,3),max_df=0.85)),
                     ('clf',LinearSVC(max_iter=2000))])


#Performing KFold cross validation
kf = KFold(n_splits=10,shuffle=True)

for train_idx,test_idx in kf.split(X_jd,y_jd):
  X_trainjd,X_testjd = X_jd.iloc[train_idx],X_jd.iloc[test_idx]
  y_trainjd,y_testjd = y_jd.iloc[train_idx],y_jd.iloc[test_idx]
  #Train the model
  print("Training model 2...")
  pipeline2.fit(X_trainjd, y_trainjd)

  #Predict Title
  pred_jt = pipeline2.predict(X_testjd)


In [ ]:
# Upload test file 'Test_file.xlsx' for prediction
uploaded = files.upload()

In [ ]:
#Run this cell to make predictions for the test file
############################ Predict from test file ################################
df_test = pd.read_excel(r'Test_file.xlsx')

#Predict Job Title from Description using pipeline2
jt = pipeline2.predict(df_test['Description'])

#Predict Skills from Job Title using pipeline1
#apply inverse transform method to retrieve predicted text labels
skills = mulbi.inverse_transform(pipeline1.predict(jt))

#Arraning Data in final DataFrame
df_fin = pd.DataFrame(columns=['Description','Title','Tags'])
df_fin['Description'] = df_test['Description']
df_fin['Title'] = jt.copy()

#Populate Skills column using the predicted skills
#Predicted skills are in a list, so unpack them in each row
df_fin['Tags'] = [x for x in skills]

#Removing paranthesis, single quotes and comma in the beginning of the string
df_fin['Tags']=df_fin['Tags'].apply(lambda x: re.sub(r'[()]','',str(x)))
df_fin['Tags']=df_fin['Tags'].apply(lambda x: re.sub(r"'",'',str(x)))
df_fin['Tags']=df_fin['Tags'].apply(lambda x: re.sub(r"^[,]",'',str(x)))

#Process final submission file to User's system
df_fin.to_excel('Final_file.xlsx',sheet_name='final',index=False)

#Setting column width for output file using openpyxl
wb = openpyxl.load_workbook('Final_file.xlsx')
sheet = wb.active
# set the width of the column 
sheet.column_dimensions['A'].width = 50
sheet.column_dimensions['B'].width = 50
sheet.column_dimensions['C'].width = 50
wb.save('Final_file.xlsx')

#Download to user's system
files.download('Final_file.xlsx') 

#Delete test file from colab to enable uploading another test file
#with different job descriptions
!rm Test_file.xlsx
